In [29]:
import os
import sys
import numpy as np
import pandas as pd
from typing import Dict
from tqdm import tqdm

import matplotlib.pyplot as plt
from ipywidgets import interact
import ipywidgets as widgets

from paths import Paths
from CardiacMotionGWAS.helpers import EnsembleGWASResults
from CardiacMotion.utils.run_helpers import Run, compute_thickness_per_aha

sys.path.append(Paths.Repos.REPOS_ROOT)

In [2]:
volume_df = get_cardiac_indices()
volume_df = volume_df.sort_index()

In [3]:
@interact
def show_volume_curve(id=widgets.Select(options=volume_df.index.get_level_values("case_id").unique()[:50])):
    
    kk = volume_df[volume_df.index.get_level_values("case_id") == id].iloc[:,0]
    plt.plot(kk.to_numpy());

interactive(children=(Select(description='id', options=(1000215, 1000336, 1000363, 1000380, 1000407, 1000434, …

In [5]:
exp_id = "3"

In [33]:
loci_df = fetch_loci_mapping()
dynamic_loci = loci_df[~loci_df.only_dynamic.isnull()].index

In [34]:
@interact
def show_volume_curve(region=widgets.Select(options=dynamic_loci), which_extreme=widgets.Checkbox()):
    
    assocs_for_region = region_assocs_df[(region_assocs_df.region == region) & (region_assocs_df.expid == "X3")].sort_values("P")
    runid, zvar = assocs_for_region.run.iloc[0], assocs_for_region.pheno.iloc[0]
    
    MLFLOW_TRACKING_URI = f"{Paths.Repos.CARDIAC_MOTION}/mlruns/"
    z_df = pd.read_csv(f"{MLFLOW_TRACKING_URI}/{exp_id}/{runid}/artifacts/latent_vector.csv")
    z_df = z_df.set_index("ID")
    z_df = z_df[zvar]
    # print()
    z_df = z_df.drop(set(ids_feos).intersection(z_df.index))
    
    ids_top = z_df[z_df > z_df.quantile(0.99)].index.to_list()
    ids_bottom = z_df[z_df < z_df.quantile(0.01)].index.to_list()
    
    if which_extreme:
        ids = set(ids_bottom).intersection(volume_df.index.get_level_values("case_id"))
    else:
        ids = set(ids_top).intersection(volume_df.index.get_level_values("case_id"))
    
    for id in list(ids)[:50]:        
        kk = volume_df[volume_df.index.get_level_values("case_id") == id].iloc[:,0]
        if len(kk) != 50:
            continue
        plt.plot(kk.to_numpy()/kk.iloc[0]);

interactive(children=(Select(description='region', options=('chr1_5', 'chr1_107', 'chr1_118', 'chr3_28', 'chr3…

In [25]:
ids = [x[:7] for x in os.listdir(f"{Paths.Repos.CARDIAC_MOTION}/notebooks/thicknesses/")]
aha_thickness_lst = []

for id in tqdm(ids):
    try:
      aha_thickness = np.load(f"{Paths.Repos.CARDIAC_MOTION}/notebooks/thicknesses/{id}_thickness_per_aha.npy")
      aha_thickness = pd.DataFrame(aha_thickness).reset_index().rename({"index": "timeframe"}, axis=1).assign(subject_id=lambda x: id).set_index(["subject_id", "timeframe"])
      aha_thickness_lst.append(aha_thickness)
    except FileNotFoundError:
        pass
    
aha_thickness_df = pd.concat(aha_thickness_lst)
aha_thickness_df.to_csv("aha_thickness.csv", index=True)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 60001/60001 [02:10<00:00, 459.81it/s]


In [74]:
aha_thickness_df.index.get_level_values("subject_id") == 

Index(['4305589', '4305589', '4305589', '4305589', '4305589', '4305589',
       '4305589', '4305589', '4305589', '4305589',
       ...
       '5733702', '5733702', '5733702', '5733702', '5733702', '5733702',
       '5733702', '5733702', '5733702', '5733702'],
      dtype='object', name='subject_id', length=3000000)

In [82]:
@interact
def show_thickness_curve(
    region=widgets.Select(options=dynamic_loci), which_extreme=widgets.Checkbox(),
    aha_segment=widgets.IntSlider(min=0, max=17)
):
    
    assocs_for_region = region_assocs_df[(region_assocs_df.region == region) & (region_assocs_df.expid == "X3")].sort_values("P")
    runid, zvar = assocs_for_region.run.iloc[0], assocs_for_region.pheno.iloc[0]
    
    MLFLOW_TRACKING_URI = f"{os.environ['HOME']}/01_repos/CardiacMotion/mlruns/"
    z_df = pd.read_csv(f"{MLFLOW_TRACKING_URI}/{exp_id}/{runid}/artifacts/latent_vector.csv")
    z_df = z_df.set_index("ID")
    z_df = z_df[zvar]
    # print()
    z_df = z_df.drop(set(ids_feos).intersection(z_df.index))
    
    ids_top = z_df[z_df > z_df.quantile(0.99)].index.to_list()
    ids_bottom = z_df[z_df < z_df.quantile(0.01)].index.to_list()
    
    if which_extreme:        
        ids = set(ids_bottom).intersection(aha_thickness_df.index.get_level_values("subject_id").unique().astype(int))
    else:
        ids = set(ids_top).intersection(aha_thickness_df.index.get_level_values("subject_id").unique().astype(int))
    
    # volume_df[volume_df.index.get_level_values("case_id") == id].iloc[:,0]
    for id in list(ids)[:10]: 
        # print(aha_thickness_df.index.get_level_values("subject_id") == str(id))
        kk = aha_thickness_df[aha_thickness_df.index.get_level_values("subject_id") == str(id)].iloc[:,aha_segment]
        # print(kk)
        if len(kk) != 50:
            continue
        plt.plot(kk.to_numpy()/kk.iloc[0]);
        plt.ylim([0.9, 3])

interactive(children=(Select(description='region', options=('chr1_5', 'chr1_107', 'chr1_118', 'chr3_28', 'chr3…

In [11]:
normalized_vols = volume_df.groupby("case_id").transform(lambda x: x/x.iloc[0])
t_above_1 = (normalized_vols["LVEDV (mL)"] > 1).groupby("case_id").aggregate(sum)
ids_feos = t_above_1[t_above_1 > 5].index
ids_feos

Int64Index([1000590, 1000892, 1001629, 1003065, 1003814, 1005995, 1006929,
            1007469, 1013023, 1013345,
            ...
            4986456, 4987449, 4987912, 4988345, 4988483, 4989669, 4989728,
            4990164, 4992639, 4994894],
           dtype='int64', name='case_id', length=3530)

In [21]:
@interact
def show_volume_curve(id=widgets.Select(options=ids_feos)):
    
    kk = volume_df[volume_df.index.get_level_values("case_id") == id].iloc[:,0]
    plt.plot(kk.to_numpy());

interactive(children=(Select(description='id', options=(1017591, 1098773, 1165317, 1191279, 1328149, 1452949, …